In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from cabm import post_processing as ppr

In [2]:
# Figure setup
# Set the font to Courier
mpl.rcParams['font.family'] = 'Courier'

# Set the line weight to thin
mpl.rcParams['lines.linewidth'] = 0.75

# Set the color scheme to black and white
mpl.rcParams['text.color'] = 'black'
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['xtick.color'] = 'black'
mpl.rcParams['ytick.color'] = 'black'
mpl.rcParams['axes.edgecolor'] = 'black'

In [3]:
agent_df = pd.read_pickle('agent_output_ad_switch_no_increment.pkl')

In [4]:
agent_df = ppr.add_date_column(agent_df)

In [5]:
def calculate_units(
    agent_df: pd.DataFrame,
    baseline_units_column: str = "Baseline_Units",
    units_to_purchase_column: str = "Units_to_Purchase",
    incremental_promo_units_column: str = "Incremental_Promo_Units",
    incremental_ad_units_column: str = "Incremental_Ad_Units",
    decremental_units_column: str = "Decremental_Units",
    brand_column: str = "Brand_Choice"
) -> pd.DataFrame:
    """
    Computes the sum of various unit columns for each brand purchased, grouped by Date and Step, and pivots the result.

    Parameters:
    agent_df (pd.DataFrame): The agent level data with 'Brand_Choice', 'Baseline_Units', 'Units_To_Purchase', 
                             'Incremental_Promo_Units', 'Incremental_Ad_Units', 'Decremental_Units', 'Date', and 'Step' columns.
    baseline_units_column (str): The column name for baseline units. Default is 'Baseline_Units'.
    units_to_purchase_column (str): The column name for units to purchase. Default is 'Units_To_Purchase'.
    incremental_promo_units_column (str): The column name for incremental promo units. Default is 'Incremental_Promo_Units'.
    incremental_ad_units_column (str): The column name for incremental ad units. Default is 'Incremental_Ad_Units'.
    decremental_units_column (str): The column name for decremental units. Default is 'Decremental_Units'.
    brand_column (str): The column name for brand choice. Default is 'Brand_Choice'.

    Returns:
    pd.DataFrame: A DataFrame with the sum of various unit columns for each brand, grouped by Date and Step, pivoted to have a column for each brand.
    """
    # Initialize a list to store the results
    units_data = []

    # Iterate over the DataFrame rows
    for index, row in agent_df.iterrows():
        brand = row[brand_column]
        date = row["Date"]
        step = index[0]  # Accessing the first level of the multiindex
        baseline_units = row[baseline_units_column]
        units_to_purchase = row[units_to_purchase_column]
        incremental_promo_units = row[incremental_promo_units_column]
        incremental_ad_units = row[incremental_ad_units_column]
        decremental_units = row[decremental_units_column]
        
        units_data.append({
            "Brand": brand,
            "Date": date,
            "Step": step,
            "Baseline_Units": baseline_units,
            "Units_to_Purchase": units_to_purchase,
            "Incremental_Promo_Units": incremental_promo_units,
            "Incremental_Ad_Units": incremental_ad_units,
            "Decremental_Units": decremental_units
        })

    # Create a new DataFrame with the units data for each brand
    units_df = pd.DataFrame(units_data)

    # Group by 'Date', 'Step', and 'Brand' and sum the units
    units_df = (
        units_df.groupby(["Date", "Step", "Brand"])
        .sum()
        .reset_index()
    )

    # Pivot the DataFrame to have a column for each brand's units
    pivot_df = units_df.pivot_table(
        index=["Date", "Step"],
        columns="Brand",
        values=["Baseline_Units", "Units_to_Purchase", "Incremental_Promo_Units", "Incremental_Ad_Units", "Decremental_Units"],
        fill_value=0
    ).reset_index()

    # Flatten the column MultiIndex
    pivot_df.columns.name = None
    pivot_df.columns = [f"{col[0]}_{col[1]}" if col[1] else col[0] for col in pivot_df.columns]

    return pivot_df

In [6]:
all_units = calculate_units(agent_df)

In [7]:
all_units

,Date,Step,Baseline_Units_A,Baseline_Units_B,Decremental_Units_A,Decremental_Units_B,Incremental_Ad_Units_A,Incremental_Ad_Units_B,Incremental_Promo_Units_A,Incremental_Promo_Units_B,Units_to_Purchase_A,Units_to_Purchase_B
0,2021-01-03,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-10,1,27.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,34.0
2,2021-01-17,2,192.0,204.0,0.0,0.0,0.0,0.0,0.0,0.0,192.0,204.0
3,2021-01-24,3,333.0,351.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,321.0
4,2021-01-31,4,423.0,456.0,0.0,0.0,0.0,0.0,0.0,0.0,392.0,427.0
...,...,...,...,...,...,...,...,...,...,...,...,...
97,2022-11-13,97,561.0,402.0,0.0,0.0,0.0,0.0,0.0,0.0,439.0,326.0
98,2022-11-20,98,552.0,335.0,0.0,0.0,0.0,0.0,0.0,0.0,470.0,282.0
99,2022-11-27,99,531.0,409.0,0.0,0.0,0.0,0.0,0.0,0.0,448.0,354.0
100,2022-12-04,100,520.0,402.0,0.0,0.0,0.0,0.0,0.0,0.0,488.0,374.0


In [8]:
import plotly.graph_objects as go

def plot_waterfall_for_brand(units_df: pd.DataFrame, brand: str, date: str):
    """
    Returns a Plotly Waterfall chart for a given brand and date.

    Parameters:
    units_df (pd.DataFrame): The DataFrame output from calculate_units function.
    brand (str): The brand for which to plot the waterfall chart.
    date (str): The date for which to plot the waterfall chart.

    Returns:
    go.Figure: A Plotly Figure object representing the waterfall chart.
    """
    # Filter the DataFrame for the given date
    date_df = units_df[units_df['Date'] == date]

    if date_df.empty:
        print(f"No data available for the date: {date}")
        return None

    # Define the column names for the brand
    baseline_units_col = f"Baseline_Units_{brand}"
    incremental_promo_units_col = f"Incremental_Promo_Units_{brand}"
    incremental_ad_units_col = f"Incremental_Ad_Units_{brand}"
    decremental_units_col = f"Decremental_Units_{brand}"
    units_to_purchase_col = f"Units_to_Purchase_{brand}"

    # Check if the brand columns exist in the DataFrame
    if not all(col in date_df.columns for col in [baseline_units_col, incremental_promo_units_col, incremental_ad_units_col, decremental_units_col, units_to_purchase_col]):
        print(f"Data for brand {brand} is not available.")
        return None

    # Extract the values for the brand
    values = date_df[[baseline_units_col, incremental_promo_units_col, incremental_ad_units_col, decremental_units_col]].iloc[0].values
    net_value = date_df[units_to_purchase_col].iloc[0]

    # Calculate the discrepancy
    calculated_units = values[0] + values[1] + values[2] - values[3]
    discrepancy = net_value - calculated_units

    # Create the waterfall chart
    fig = go.Figure(go.Waterfall(
        name=brand,
        orientation="v",
        measure=["relative", "relative", "relative", "relative", "total"],
        x=["Baseline Units", "Incremental Promo Units", "Incremental Ad Units", "Decremental Units", "Units to Purchase"],
        y=[values[0], values[1], values[2], -values[3], net_value],
        connector={"line": {"color": "rgb(63, 63, 63)"}}
    ))

    # Add discrepancy if it exists
    if discrepancy != 0:
        fig.add_trace(go.Waterfall(
            name="Discrepancy",
            orientation="v",
            measure=["relative"],
            x=["Discrepancy"],
            y=[discrepancy],
            connector={"line": {"color": "rgb(63, 63, 63)"}}
        ))

    fig.update_layout(
        title=f"Waterfall Chart for {brand} on {date}",
        waterfallgap=0.3
    )

    return fig

# Example usage:
# fig = plot_waterfall_for_brand(all_units, 'A', '2021-01-10')
# if fig:
#     fig.show()

In [11]:

fig = plot_waterfall_for_brand(all_units, 'A', '2022-01-02')
if fig:
    fig.show()

Values: [529.   0.   0.   0.]
Net Value: 483.0
Calculated Units: 529.0
Discrepancy: -46.0


In [10]:
def plot_waterfall_for_brand(units_df: pd.DataFrame, brand: str, date: str):
    """
    Returns a Plotly Waterfall chart for a given brand and date.

    Parameters:
    units_df (pd.DataFrame): The DataFrame output from calculate_units function.
    brand (str): The brand for which to plot the waterfall chart.
    date (str): The date for which to plot the waterfall chart.

    Returns:
    go.Figure: A Plotly Figure object representing the waterfall chart.
    """
    # Filter the DataFrame for the given date
    date_df = units_df[units_df['Date'] == date]

    if date_df.empty:
        print(f"No data available for the date: {date}")
        return None

    # Define the column names for the brand
    baseline_units_col = f"Baseline_Units_{brand}"
    incremental_promo_units_col = f"Incremental_Promo_Units_{brand}"
    incremental_ad_units_col = f"Incremental_Ad_Units_{brand}"
    decremental_units_col = f"Decremental_Units_{brand}"
    units_to_purchase_col = f"Units_to_Purchase_{brand}"

    # Check if the brand columns exist in the DataFrame
    if not all(col in date_df.columns for col in [baseline_units_col, incremental_promo_units_col, incremental_ad_units_col, decremental_units_col, units_to_purchase_col]):
        print(f"Data for brand {brand} is not available.")
        return None

    # Extract the values for the brand
    values = date_df[[baseline_units_col, incremental_promo_units_col, incremental_ad_units_col, decremental_units_col]].iloc[0].values
    net_value = date_df[units_to_purchase_col].iloc[0]

    # Calculate the discrepancy
    calculated_units = values[0] + values[1] + values[2] - values[3]
    discrepancy = net_value - calculated_units

    # Debugging: Print intermediate values
    print(f"Values: {values}")
    print(f"Net Value: {net_value}")
    print(f"Calculated Units: {calculated_units}")
    print(f"Discrepancy: {discrepancy}")

    # Round values to avoid floating-point precision issues
    calculated_units = round(calculated_units, 2)
    net_value = round(net_value, 2)
    discrepancy = round(discrepancy, 2)

    # Create the waterfall chart
    fig = go.Figure(go.Waterfall(
        name=brand,
        orientation="v",
        measure=["relative", "relative", "relative", "relative", "total"],
        x=["Baseline Units", "Incremental Promo Units", "Incremental Ad Units", "Decremental Units", "Units to Purchase"],
        y=[values[0], values[1], values[2], -values[3], net_value],
        connector={"line": {"color": "rgb(63, 63, 63)"}}
    ))

    # Add discrepancy if it exists
    if discrepancy != 0:
        fig.add_trace(go.Waterfall(
            name="Discrepancy",
            orientation="v",
            measure=["relative"],
            x=["Discrepancy"],
            y=[discrepancy],
            connector={"line": {"color": "rgb(63, 63, 63)"}}
        ))

    fig.update_layout(
        title=f"Waterfall Chart for {brand} on {date}",
        waterfallgap=0.3
    )

    return fig

In [10]:
def calculate_average_adstock(agent_df):
    '''
    Computes average adstock per brand per step from agent level output
    '''
    # Initialize empty lists to store the results
    average_adstock_A = []
    average_adstock_B = []
    dates = []
    steps = []

    # Iterate over the 'Ad_Stock' column
    for index, row in agent_df.iterrows():
        # Append the adstock for each brand to the respective list
        average_adstock_A.append(row['Adstock']['A'])
        average_adstock_B.append(row['Adstock']['B'])
        dates.append(row['Date'])
        steps.append(index[0])  # Assuming 'Step' is the first level of the index

    # Create a new DataFrame with the average adstock for each brand
    average_adstock_df = pd.DataFrame({
        'Average_Adstock_A': average_adstock_A,
        'Average_Adstock_B': average_adstock_B,
        'Date': dates,
        'Step': steps
    })

    # Group by 'Date' and 'Step' and calculate the average adstock
    average_adstock_df = average_adstock_df.groupby(['Date', 'Step']).mean().reset_index()

    return average_adstock_df

In [11]:
calculate_average_adstock(agent_df)

,Date,Step,Average_Adstock_A,Average_Adstock_B
0,2021-01-03,0,0.000000,0.0
1,2021-01-10,1,1.000000,1.0
2,2021-01-17,2,1.000000,1.0
3,2021-01-24,3,1.000000,1.0
4,2021-01-31,4,1.000000,1.0
...,...,...,...,...
199,2024-10-27,199,15644.045132,1.0
200,2024-11-03,200,14068.597643,1.0
201,2024-11-10,201,12816.123055,1.0
202,2024-11-17,202,11794.099716,1.0
